# **거래까지 걸린 시간 찾기**

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_csv("C:/Users/whileduck/Desktop/빈좌석/클래식_빈좌석포함데이터.csv", low_memory = False)

display(df.head())
print(df.shape)

,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
3,1층 A블록1열 4,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6,23,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000
4,1층 A블록1열 5,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5,24,...,120,15,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000


(405810, 43)


# **예매된 경우 거래까지 걸린 시간 찾기**

In [45]:
def convert_string_to_hours(time_string):

    parts = time_string.split()
    time_parts = parts[2].split(':')
    
    days = int(parts[0])
    hour = int(time_parts[0])
    minutes = int(time_parts[1])

    return (days * 24) + hour + (minutes / 60)

def transaction_time(data):
    
    data = data.copy()
    data['거래까지걸린시간'] = np.NaN # default 값은 NaN 값을 맵핑하기 
    data['거래까지걸린시간_시간'] = np.NaN # default 값은 NaN 값을 맵핑하기 
    
    
    # 조건 찾기 
    purchase_cond = data['예매여부'] == 1 # 예매 된 경우에 대한 조건 
    open_cond = data['pre_open_date'].isna() # 선 예매가 되지 않은 경우에 대한 조건 
    
    # 조건에 따라 다르게 맵핑하기 
    
    open_time = pd.to_datetime(data.loc[(purchase_cond) & (open_cond), 'open_date'])
    open_purchase_time = pd.to_datetime(data.loc[(purchase_cond) & (open_cond), '전체거래시간'])
    
    pre_open_time = pd.to_datetime(data.loc[(purchase_cond) & (~open_cond), 'pre_open_date'])
    pre_open_purchase_time = pd.to_datetime(data.loc[(purchase_cond) & (~open_cond), '전체거래시간'])
    
    data.loc[(purchase_cond) & (open_cond),'거래까지걸린시간'] = open_purchase_time - open_time
    data.loc[(purchase_cond) & (~open_cond),'거래까지걸린시간'] = pre_open_purchase_time - pre_open_time
    
    data.loc[purchase_cond, '거래까지걸린시간_시간'] = data.loc[purchase_cond,'거래까지걸린시간'].map(lambda x: convert_string_to_hours(str(x)))
    
    return data
    
    

In [48]:
result = transaction_time(df)

In [50]:
display(result.head())
display(result.tail())

,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량,거래까지걸린시간,거래까지걸린시간_시간
0,1층 A블록1열 1,1층,A블록,1,1,1451,542,-93,1층 E블록1열 9,21,...,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1
1,1층 A블록1열 2,1층,A블록,1,2,1406,555,-93,1층 E블록1열 8,22,...,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1
2,1층 A블록1열 3,1층,A블록,1,3,1361,568,-93,1층 E블록1열 7,23,...,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1
3,1층 A블록1열 4,1층,A블록,1,4,1315,580,-93,1층 E블록1열 6,23,...,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1
4,1층 A블록1열 5,1층,A블록,1,5,1270,591,-93,1층 E블록1열 5,24,...,N,2018,11,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1000,19 days 15:06:00,471.1


,seat,층,블록,열,넘버,X,Y,Z,대칭점,무대_밑_좌우시야각,...,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,누적검색량,거래까지걸린시간,거래까지걸린시간_시간
405805,2층 BOX6 2,2층,BOX,6,2,-1432,505,383,2층 BOX1 1,22,...,Y,2023,6,2023-06-06,2023-06-01,2023-06-06 19:30:00,2023-03-26 16:33:00,770,1 days 16:33:00,40.550000
405806,2층 BOX6 3,2층,BOX,6,3,-1500,565,428,2층 BOX1 4,21,...,Y,2023,6,2023-06-06,2023-06-01,2023-06-06 19:30:00,2023-03-26 16:33:00,770,1 days 16:33:00,40.550000
405807,2층 BOX6 4,2층,BOX,6,4,-1454,617,428,2층 BOX1 3,21,...,Y,2023,6,2023-06-06,2023-06-01,2023-06-06 19:30:00,2023-05-31 01:57:00,770,67 days 01:57:00,1609.950000
405808,2층 BOX6 5,2층,BOX,6,5,-1522,677,483,2층 BOX1 6,20,...,NaN,2023,6,2023-06-06,2023-06-01,2023-06-06 19:30:00,NaN,770,NaN,NaN
405809,2층 BOX6 6,2층,BOX,6,6,-1568,624,483,2층 BOX1 5,20,...,N,2023,6,2023-06-06,2023-06-01,2023-06-06 19:30:00,2023-06-02 19:10:00,770,69 days 19:10:00,1675.166667


##### **잘 계산된 것으로 보인다 저장해주자**

In [52]:
file_path = 'C:/Users/whileduck/Desktop/빈좌석/'

result.to_csv(file_path + '클래식_빈좌석포함데이터.csv',index = False)  